<a href="https://colab.research.google.com/github/SunnyZhao2004/Data_Project/blob/main/president_party.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
########### This chunk is for setup ########################

from google.colab import drive
import pandas as pd
import glob

# Mount Google Drive
drive.mount('/content/drive')

# Set the path to the shared folder on your Google Drive
data_folder = '/content/drive/My Drive/0812Fullmerged/'

# Get all Parquet files in the shared folder
all_files = glob.glob(data_folder + "*.parquet")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
##################### This is for president party merging ################
# Step 1: Load the president term data from Excel
president_terms_file = '/content/drive/My Drive/Data/president_terms.xlsx'
president_terms = pd.read_excel(president_terms_file)

# Step 2: Loop through each Parquet file, process it, and save the results
for file_path in all_files:
    # Load the Parquet file
    df = pd.read_parquet(file_path)

    # Ensure action_date is in datetime format
    df['action_date'] = pd.to_datetime(df['action_date'], format='mixed')

    # Step 1: Initialize an empty president_party column
    df['president_party'] = None

    # Step 2: Apply the president terms to the data
    for _, row in president_terms.iterrows():
        # Create a mask where action_date is between the president's start and end dates
        mask = (df['action_date'] >= row['Start_date']) & (df['action_date'] <= row['End_date'])

        # Assign the president's party based on the mask
        df.loc[mask, 'president_party'] = row['Party']

    # Get the file name from the file path (to use in output)
    file_name = file_path.split('/')[-1].replace('.parquet', '_processed.parquet')

    # Save the processed data with a new name
    output_file = f'/content/drive/My Drive/Data/{file_name}'
    df.to_parquet(output_file, index=False)

    print(f'Processed and saved file: {output_file}')

In [ ]:
import pandas as pd
import glob

# Step 1: Load the `psc_codes_by_category` file once outside the loop
psc_codes = pd.read_csv('/content/drive/My Drive/Data/psc_codes_by_category.csv')

# Create a dictionary for fast lookup of 'industry_category'
psc_dict = psc_codes.set_index('4-Digit PSC')['Category'].to_dict()

# Step 3: Loop through each Parquet file, process it, and save the results
for idx, file_path in enumerate(all_files, 1):
    # Load the Parquet file
    df = pd.read_parquet(file_path)

    # Ensure `product_or_service_code` is a string for mapping
    df['product_or_service_code'] = df['product_or_service_code'].astype(str)

    # Map `product_or_service_code` to `industry_category` using the dictionary
    df['industry_category'] = df['product_or_service_code'].map(psc_dict)

    # Step 4: Save the processed file with a new name
    output_file = f'/content/drive/My Drive/Data/0812_merged_chunk_{idx}_with_industry_category.parquet'
    df.to_parquet(output_file, index=False)

    print(f'Processed and saved file: {output_file}')


In [3]:
############ debug: set df to the first file ##############
data_folder = '/content/drive/My Drive/0812Fullmerged/0812_merged_chunk_1.parquet'
df2 = pd.read_parquet(data_folder)

In [5]:
print(df2['product_or_service_code'])

0          V231
1          Z111
2          L016
3          C211
4          R408
           ... 
7709931    5680
7709932    6220
7709933    5680
7709934    5680
7709935    5680
Name: product_or_service_code, Length: 7709936, dtype: object


In [10]:

# Step 1: Load the `psc_codes_by_category` file
psc_codes = pd.read_csv('/content/drive/My Drive/Data/psc_codes_by_category.csv' )

# Create a dictionary for fast lookup of 'industry_category'
psc_dict = psc_codes.set_index('4-Digit PSC')['Category'].to_dict()

# Ensure `product_or_service_code` is a string for mapping
df['product_or_service_code'] = df['product_or_service_code'].astype(str)

# Step 4: Map `product_or_service_code` to `industry_category` using the dictionary
df['industry_category'] = df['product_or_service_code'].map(psc_dict)

# Create a new folder in your Google Drive (e.g., "ProcessedFiles")
output_file = '/content/drive/My Drive/Data/0812_merged_chunk_1_with_industry_category.parquet'
df.to_parquet(output_file, index=False)

print(f'Processed and saved file: {output_file}')


Processed and saved file: /content/drive/My Drive/Data/0812_merged_chunk_1_with_industry_category.parquet


In [3]:
df = pd.read_parquet('/content/drive/My Drive/Data/0812_merged_chunk_1_with_industry_category.parquet')

In [6]:
import numpy as np
num_nan_values = np.sum(df['industry_category'].isnull())
print(num_nan_values)

615609
